In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
model_id = 'DNN-hpsearch-easy'

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd

from annsa.template_sampling import *
from annsa.load_pretrained_network import save_features

In [ ]:
from hyperparameter_models import make_dense_model as make_model

In [ ]:
import tensorflow.contrib.eager as tfe

In [ ]:
tf.enable_eager_execution()

#### Import model, training function 

In [ ]:
from annsa.model_classes import (dnn_model_features,
                                 DNN,
                                 save_model,
                                 train_earlystop)

## Load testing dataset 

In [ ]:
dataset = np.load('../dataset_generation/hyperparametersearch_dataset_100_easy.npy')

In [ ]:
all_spectra = np.float64(np.add(dataset.item()['sources'], dataset.item()['backgrounds']))
all_keys = dataset.item()['keys']

mlb=LabelBinarizer()

all_keys_binarized = mlb.fit_transform(all_keys)

# Train network

### Define hyperparameters

In [ ]:
number_hyperparameters_to_search = 256
earlystop_errors_test = []

### Search hyperparameters

In [ ]:
skf = StratifiedKFold(n_splits=5, random_state=5)
testing_errors = []
all_kf_errors = []

for network_id in range(number_hyperparameters_to_search):
    print(network_id)
    model, model_features = make_model(all_keys_binarized)
    filename = os.path.join('hyperparameter-search-results',
                            model_id + '-' +str(hyperparameter_index))
    save_features(model_features, filename)
    
    k_folds_errors = []
    for train_index, test_index in skf.split(all_spectra, all_keys):
        # reset model on each iteration
        model = DNN(model_features)
        optimizer = tf.train.AdamOptimizer(model_features.learining_rate)

        costfunction_errors_tmp, earlystop_errors_tmp = train_earlystop(
                training_data=all_spectra[train_index],
                training_keys=all_keys_binarized[train_index],
                testing_data=all_spectra[test_index],
                testing_keys=all_keys_binarized[test_index],
                model=model,
                optimizer=optimizer,
                num_epochs=200,
                obj_cost=model.cross_entropy,
                earlystop_cost_fn=model.f1_error,
                earlystop_patience=10,
                not_learning_patience=10,
                not_learning_threshold=0.9,
                verbose=True,
                fit_batch_verbose=10,
                data_augmentation=model.default_data_augmentation)
        k_folds_errors.append(earlystop_errors_tmp)
        all_kf_errors.append(earlystop_errors_tmp)

    testing_errors.append(np.average(k_folds_errors))
    np.save('./final-models/final_test_errors_'+model_id, testing_errors)
    np.save('./final-models/final_kf_errors_'+model_id, all_kf_errors)
    network_id += 1 